In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from operator import itemgetter
import matplotlib as mpl
import matplotlib.pyplot as plt
from timeit import time
import itertools # para generar las permutaciones del exhaustivo

capitales = pd.read_csv('capitales.csv')

# ------------------------ formateo de tabla de distancias
tabla_distancias = pd.read_csv('distancias_capitales.csv')
tabla_distancias = tabla_distancias.rename(columns={'Distancias en kilómetros': 'Capital'})
columnas = ['capital']
for i in range(24):
    columnas.append(i)
tabla_distancias.columns = columnas
# ------------------------

In [2]:
capitales=pd.read_excel('CapitalesArg.xlsx', index_col=None, header=None)
a=[]
a=list(capitales[1][1:24])
b=[]
for i in range(len(a)):
    if type(a[i])!=float:
        b.append(int(a[i]))
print(b[:])
print(str(np.min(b)))
min=str(np.min(b))
recorrido=[]
capis=capitales[1][1:24]
for id, cap in enumerate (capis):
                if cap==min:
                    recorrido.append(str(id+1))
print(recorrido)

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

In [ ]:
#esta mal
def heuristicoA(tabla_capitales, nro_ciudad):
    recorrido=[]
    recorrido.append(str(nro_ciudad))
    elegida=nro_ciudad
    for nro_capi in range (1, 22):
        band=False
        while (band==False):
            a, b=[],[]
            a=list(tabla_capitales[elegida][1:24])
            for i in range(len(a)):
                if type(a[i])!=float:
                    b.append(int(a[i]))
            min=str(np.min(b))
            for id, cap in enumerate (tabla_capitales):
                if cap==min:
                    recorrido.append(str(id+1))
            #band=True
            for i in range(len(recorrido)-1):
                if i==len(recorrido)-2:
                    band=True
                if recorrido[i]==recorrido[-1]:
                    recorrido.pop()
                    band=False
                    break
        elegida=int(recorrido[-1])            
    recorrido.append(recorrido[0])
    return recorrido

In [125]:
def heuristicoA(tabla_distancias, nro_ciudad):
    recorrido = []
    recorrido.append(nro_ciudad)
    distancia_recorrida = 0
    
    id_capital_sig = nro_ciudad #
    tabla_auxiliar = tabla_distancias[id_capital_sig].copy() #inicializar tabla auxiliar

    while len(recorrido) < len(tabla_distancias):  # len(tabla_distancias) -> 24
    
        distancia_mas_cercana = tabla_auxiliar.min()
        id_capital_sig = tabla_auxiliar.idxmin()
        
        if id_capital_sig in recorrido:  # si verdadero, esa capital ya se visitó y por lo tanto se elimina de tabla auxiliar
            tabla_auxiliar = tabla_auxiliar.drop(id_capital_sig) 
        else:
            distancia_recorrida += distancia_mas_cercana  
            recorrido.append(id_capital_sig)
            tabla_auxiliar = tabla_distancias[id_capital_sig].copy() # reinicio la tabla auxiliar para que tenga todas las filas

    return recorrido, distancia_recorrida

In [2]:
def formatear(tabla_capitales, recorrido):
    # devuelve una tabla con latitudes y longitudes, ordenada por orden de recorrido lista para ser impresa en el mapa
    tabla_final = pd.DataFrame(columns=['capital', 'latitud', 'longitud'])
    
    for i in range(len(recorrido)):
        tabla_final = tabla_final.append(tabla_capitales.iloc[recorrido[i]])
        
    return tabla_final

In [131]:
# para probar heuristicoA
recorrido, distancia_recorrida = heuristicoA(tabla_distancias, 5)
s = formatear(capitales, recorrido)
s

,capital,latitud,longitud
5,La Rioja,-29.416018,-66.872266
13,S.F.d.V.d. Catamarca,-28.470552,-65.785805
21,Sgo. del Estero,-27.803519,-64.268081
14,S.M. de Tucumán,-26.824140,-65.222600
16,Salta,-24.793299,-65.413140
15,S.S. de Jujuy,-24.192795,-65.298883
11,Resistencia,-27.426056,-58.988848
2,Corrientes,-27.483745,-58.812996
3,Formosa,-26.181290,-58.190586
9,Posadas,-27.382250,-55.920130


In [135]:
s.iloc[0]['capital']

'La Rioja'

In [ ]:
#no esta probado
def heuristicoB(tabla_capitales):
    recorridos=[]
    for i in range (24):
        recorridos.append(heuristicoA(tabla_capitales, i+1))
    longitudes=[]
    for cromosoma in recorridos:
        id=0
        for prov in cromosoma:
            if id<23:
                kms=int(tabla_capitales[int(prov)][int(cromosoma[id+1])])+kms
                id=id+1
        kms=kms+int(tabla_capitales[int(cromosoma[-2])][int(cromosoma[-1])])
        longitudes.append(kms)
    for id, cromosoma in recorridos:
        aux.append(cromosoma, longitides[id])
    aux = sorted(aux, key=itemgetter(1), reverse=True)
    mejor_recorrido=aux[0][0]
    kms=aux[0][1]
    return mejor_recorrido, kms

# con alg. geneticos

In [4]:
def inicializar(tamaño_poblacion, longitud_cromosoma):
    poblacion_inicial = []
    for i in range(tamaño_poblacion):
        num = []
        num.append(str(np.random.randint(1,25)))
        for i in range(1, longitud_cromosoma-1):
            band=True
            while(band==True):
                prov=str(np.random.randint(1,25))                
                repetido=0
                for j in (num):
                    if prov==j:
                        repetido=repetido+1
                if repetido<1:
                    num.append(prov)
                    band=False
        num.append(num[0])
        poblacion_inicial.append(num)
    
    return poblacion_inicial

In [75]:
def mutacion(cromosoma):
    provA = np.random.randint(1, len(cromosoma)-1)
    provB = np.random.randint(1, len(cromosoma)-1)
    #no se si haria falta validar que no toque la misma provincia a permutar
    band=False
    while (band==False):
        if provA==provB:
            provB = np.random.randint(1, len(cromosoma)-1)
        else:
            band=True
    A=cromosoma[provA]
    B=cromosoma[provB]
    cromosoma[provA]=B
    cromosoma[provB]=A
    return cromosoma

In [119]:
print(type(int(inicializar(5, 24)[0][0])))

<class 'int'>


In [12]:
#anda pero no es el que pide
def crossover(cromosoma):
    cromosoma_result=[]
    punto_corte = np.random.randint(1, len(cromosoma)-1)
    p1 = cromosoma[1:punto_corte]
    p2 = cromosoma[punto_corte:len(cromosoma)]
    cromosoma_result=list(cromosoma[0])+p2+p1+list(cromosoma[len(cromosoma)-1])
    return cromosoma_result

In [7]:
def crossover_ciclico(padre1, padre2):
    hijo1 = []
    hijo2 = []
    for i in range(len(padre1)):
        hijo1.append('0')
        hijo2.append('0')
        
    # HIJO 1
    valor_inicial = padre1[0]
    hijo1[0] = valor_inicial
    gen = padre2[0]
 
    i = 1
    while True:
        if padre1[i] == gen:
            hijo1[i] = gen
            gen = padre2[i]
            if valor_inicial == padre2[i]:
                break
        if i >= len(padre1)-1:
            i = 1
        else:
            i += 1
    #intercambiar los que quedaron vacíos
    for i, prov in enumerate(hijo1):
        if prov == '0':
            hijo1[i] = padre2[i]
    hijo1.append(valor_inicial)
    
    # HIJO 2
    valor_inicial = padre2[0]
    hijo2[0] = valor_inicial
    gen = padre1[0]
 
    i = 1
    while True:  
        if padre2[i] == gen:
            hijo2[i] = gen
            gen = padre1[i]
            if valor_inicial == padre1[i]:
                break
        if i >= len(padre1)-1:
            i = 1
        else:
            i += 1
    
    #intercambiar los que quedaron vacíos
    for i, prov in enumerate(hijo2):
        if prov == '0':
            hijo2[i] = padre1[i]
    hijo2.append(valor_inicial)
    
    return hijo1, hijo2

In [ ]:
# PARA PROBAR EL CROSSOVER:
p1 = [9,8,2,1,7,4,5,10,6,3]
p2 = [1,2,3,4,5,6,7,8,9,10]
h1, h2 = crossover_ciclico(p1,p2)
print(h1)
print(h2)

In [ ]:
poblacion=inicializar(5, 24)
hijo1, hijo2= crossover_ciclico(poblacion[1], poblacion[3])
print(hijo1)
print(hijo2)
print(len(hijo1))

In [27]:
def func_objetivo(poblacion, distancias):
    f_objetivo=0
    #sumatoria de todas las distancias de todas las poblaciones
    for cromosoma in poblacion:
        id=0
        for prov in cromosoma:
            if id < len(cromosoma):
                f_objetivo=int(distancias[int(prov)][int(cromosoma[id+1])])+f_objetivo
                id=id+1
        f_objetivo=f_objetivo+int(distancias[int(cromosoma[-2])][int(cromosoma[-1])])          
    return f_objetivo

In [26]:
def func_objetivo(poblacion, distancias)
    f_objetivo = 0
    
    for cromosoma in poblacion:
        for i in range(len(cromosoma)):
            f_objetivo = 
            
    
    return f_objetivo

IndexError: string index out of range

In [20]:
tabla_distancias

,capital,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,Cdad. de Bs. As.,NaN,646.0,792.0,933.0,53.0,986.0,985.0,989.0,375.0,...,1080.0,1334.0,1282.0,1005.0,749.0,393.0,579.0,939.0,2373.0,799.0
1,Córdoba,646.0,NaN,677.0,824.0,698.0,340.0,466.0,907.0,348.0,...,517.0,809.0,745.0,412.0,293.0,330.0,577.0,401.0,2618.0,1047.0
2,Corrientes,792.0,677.0,NaN,157.0,830.0,814.0,1131.0,1534.0,500.0,...,633.0,742.0,719.0,1039.0,969.0,498.0,1136.0,535.0,3131.0,1527.0
3,Formosa,933.0,824.0,157.0,NaN,968.0,927.0,1269.0,1690.0,656.0,...,703.0,750.0,741.0,1169.0,1117.0,654.0,1293.0,629.0,3284.0,1681.0
4,La Plata,53.0,698.0,830.0,968.0,NaN,1038.0,1029.0,1005.0,427.0,...,1132.0,1385.0,1333.0,1053.0,795.0,444.0,602.0,991.0,2350.0,789.0
5,La Rioja,986.0,340.0,814.0,927.0,1038.0,NaN,427.0,1063.0,659.0,...,330.0,600.0,533.0,283.0,435.0,640.0,834.0,311.0,2821.0,1311.0
6,Mendoza,985.0,466.0,1131.0,1269.0,1029.0,427.0,NaN,676.0,790.0,...,756.0,1023.0,957.0,152.0,235.0,775.0,586.0,713.0,2435.0,1019.0
7,Neuquén,989.0,907.0,1534.0,1690.0,1005.0,1063.0,676.0,NaN,1053.0,...,1370.0,1658.0,1591.0,824.0,643.0,1049.0,422.0,1286.0,1762.0,479.0
8,Paraná,375.0,348.0,500.0,656.0,427.0,659.0,790.0,1053.0,NaN,...,707.0,959.0,906.0,757.0,574.0,19.0,642.0,566.0,2635.0,1030.0
9,Posadas,834.0,919.0,291.0,263.0,857.0,1098.0,1384.0,1709.0,658.0,...,924.0,1007.0,992.0,1306.0,1200.0,664.0,1293.0,827.0,3207.0,1624.0


In [138]:
def fitness(cromosoma, f_obj, distancias):
    #fitness=(f_obj-recorrido)/f_obj
    recorrido=0
    id=0
    for prov in cromosoma:
        if id<23:
            recorrido=int(distancias[int(prov)][int(cromosoma[id+1])])+recorrido
            id=id+1
    recorrido=recorrido+int(distancias[int(cromosoma[-2])][int(cromosoma[-1])])
    fitness=(f_obj-recorrido)/f_obj
    return fitness

In [166]:
a=inicializar(5,24)
print(a[0])
f_obj=func_objetivo(inicializar(5, 24), capitales)
print(f_obj)
print(fitness(a[4], f_obj, capitales))

['18', '2', '19', '21', '15', '6', '13', '22', '20', '5', '23', '9', '11', '12', '4', '3', '17', '7', '10', '24', '16', '1', '8', '18']
139417
0.7989915146646391


In [9]:
#no esta probado
def seleccionar(poblacion, distancias, elitismo = False):
    lista_poblacion = []
    id_seleccionado = 0
    padres = []
    fobj=func_objetivo(poblacion, distancias)
    for id, individuo in enumerate(poblacion):
        lista_poblacion.append([id, individuo, fitness(individuo, fobj, distancias)])
        
    lista_poblacion = sorted(lista_poblacion, key=itemgetter(2), reverse=True)
    
    if elitismo:
        padres.append(lista_poblacion[0][1])
        padres.append(lista_poblacion[1][1])
    else:
        # RULETA 
        limite_superior = np.sum([x[2] for x in lista_poblacion]) # suma de las fitness

        for k in range(2):
            num_aleatorio = np.random.rand() * limite_superior    
            fitness_acumulada = lista_poblacion[0][2]
            i = 0
            
            while(fitness_acumulada < limite_superior):
                if num_aleatorio <= fitness_acumulada:
                    id_seleccionado = lista_poblacion[i][0]
                    break
                else: 
                    i += 1
                    fitness_acumulada += lista_poblacion[i][2]
                    

            if fitness_acumulada == limite_superior:
                id_seleccionado = lista_poblacion[-1][0]

            for sublista in lista_poblacion:
                if sublista[0] == id_seleccionado:
                    cromosoma_elegido = sublista[1]
                    break
            padres.append(cromosoma_elegido)
            
    return padres[0], padres[1]

In [ ]:
#ver y/o arreglar generar_estadisticos() y main()

In [ ]:
#no esta checkeado ni probado
def generar_estadisticos(poblaciones, elitismo, distancias):
    # -----------------------------------------------------------------------------------------
    # GENERACIÓN DE TABLA
    df = pd.DataFrame(columns=['Poblacion', 'Mínimo', 'Máximo', 'Media', 'Mediana', 'Desv.Estándar', 'Mejor cromosoma'])
    minimos, maximos, medias, medianas, desviaciones, mejores_cromosomas = [], [], [], [], [], []
    ids = [x for x in range(len(poblaciones))]
    
    for poblacion in poblaciones:
        puntajes = []
        for individuo in poblacion:
            puntajes.append(f_objetivo(poblaciones, distancias))

        minimos.append(np.min(puntajes))
        maximos.append(np.max(puntajes))
        medias.append(np.mean(puntajes))
        medianas.append(np.median(puntajes))
        desviaciones.append(np.std(puntajes))
        
        max_puntaje = 0
        id_puntaje = 0
        for i, puntaje in enumerate(puntajes):
            if puntaje > max_puntaje:
                max_puntaje = puntaje
                id_puntaje = i
        mejores_cromosomas.append(poblacion[id_puntaje])
        
    df['Poblacion'] = ids
    df['Mínimo'] = minimos
    df['Máximo'] = maximos
    df['Media'] = medias
    df['Mediana'] = medianas
    df['Desv.Estándar'] = desviaciones
    df['Mejor cromosoma'] = mejores_cromosomas
    df = df.set_index('Poblacion')
    df = df.round(decimals = 3)
    
    # ----------------------------------------------------------------------------------------------
    # GENERACIÓN DE GRÁFICOS
    fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize = (13, 8), sharex = True)

    ax0.plot(ids, medias, label='Media', linewidth=1.7, color="firebrick")
    ax1.plot(ids, minimos, label='Mínimo', linewidth=1.7, color="blue")
    ax2.plot(ids, maximos, label='Máximo', linewidth=1.7, color="green")

    ax0.set_title(f'Medias         ---         Valor final: {medias[-1]:2.4f}') 
    ax0.set_ylabel('Media')
    ax0.set_xlim(0,len(ids) - 1)
    ax0.grid(True)
    ax0.legend(loc='lower right')
    ax0.tick_params(axis='both', which='major', labelsize=12)
    ax0.tick_params(axis='both', which='minor', labelsize=12)
    
    ax1.set_title(f'Mínimos         ---         Valor final: {minimos[-1]:2.4f}') 
    ax1.set_ylabel('Mínimo')
    ax1.set_xlim(0,len(ids) - 1)
    ax1.grid(True)
    ax1.legend(loc='lower right')
    ax1.tick_params(axis='both', which='major', labelsize=12)
    ax1.tick_params(axis='both', which='minor', labelsize=12)
    
    
    ax2.set_title(f'Máximos         ---         Valor final: {maximos[-1]:2.4f}') 
    ax2.set_ylabel('Máximo')
    ax2.set_xlim(0,len(ids) - 1)
    ax2.set_xlabel('Número de generación')
    ax2.grid(True)
    ax2.legend(loc='lower right')
    ax2.tick_params(axis='both', which='major', labelsize=12)
    ax2.tick_params(axis='both', which='minor', labelsize=12)
    
    if ids[-1] <= 20:    
        ax2.set_xticks(ids)
    
    
    plt.rcParams.update({'font.size': 10})
    plt.tight_layout()
    
    
    plt.show()
    
    return df

In [16]:
#no esta probado
#PASAR 2 Y 4 CROMOSOMAS EN ELITISMO. NO HACE FALTA HACER GRÁFICOS SI HACEMOS MAPAS
def main(distancias, p_crossover=0.9, p_mutacion=0.2, ciclos=200, tamaño_poblacion=50, longitud_cromosoma=24, elitismo=True):
    poblacion_actual = inicializar(tamaño_poblacion, longitud_cromosoma)
    poblaciones = []
    poblaciones.append(poblacion_actual)
    cantidad_selecciones = tamaño_poblacion // 2
    ciclo_convergencia = 0
    
    for i in range(ciclos):
        poblacion_nueva = []
        
        if elitismo:
            individuo_elite_1, individuo_elite_2 = seleccionar(poblacion_actual, distancias, elitismo = True)
            poblacion_nueva.append(individuo_elite_1)
            poblacion_nueva.append(individuo_elite_2)
            
        for j in range(tamaño_poblacion // 2 + 1):
         
            if len(poblacion_nueva) == tamaño_poblacion:
                continue
                
            # SELECCIONAR
            padre_1, padre_2 = seleccionar(poblacion_actual, distancias, elitismo=False)

            # CRUZAR
            if np.random.rand() <= p_crossover:
                hijo_1, hijo_2 = crossover(padre_1, padre_2)
            else: 
                hijo_1 = padre_1
                hijo_2 = padre_2

            # MUTAR
            if np.random.rand() <= p_mutacion:
                hijo_1 = mutacion(hijo_1)
            if np.random.rand() <= p_mutacion:
                hijo_2 = mutacion(hijo_2)

            # PASAR A PROXIMA GENERACION
            poblacion_nueva.append(hijo_1)
            if len(poblacion_nueva) == tamaño_poblacion:
                continue
            else:
                poblacion_nueva.append(hijo_2)

        poblacion_actual = poblacion_nueva
        
        poblaciones.append(poblacion_nueva)
        
            
    poblaciones_df = pd.DataFrame(poblaciones)
    
    resultados = generar_estadisticos(poblaciones, elitismo)
    return resultados, ciclo_convergencia

In [17]:
main(tabla_distancias)

KeyError: 24

In [15]:
tabla_distancias

,capital,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,Cdad. de Bs. As.,NaN,646.0,792.0,933.0,53.0,986.0,985.0,989.0,375.0,...,1080.0,1334.0,1282.0,1005.0,749.0,393.0,579.0,939.0,2373.0,799.0
1,Córdoba,646.0,NaN,677.0,824.0,698.0,340.0,466.0,907.0,348.0,...,517.0,809.0,745.0,412.0,293.0,330.0,577.0,401.0,2618.0,1047.0
2,Corrientes,792.0,677.0,NaN,157.0,830.0,814.0,1131.0,1534.0,500.0,...,633.0,742.0,719.0,1039.0,969.0,498.0,1136.0,535.0,3131.0,1527.0
3,Formosa,933.0,824.0,157.0,NaN,968.0,927.0,1269.0,1690.0,656.0,...,703.0,750.0,741.0,1169.0,1117.0,654.0,1293.0,629.0,3284.0,1681.0
4,La Plata,53.0,698.0,830.0,968.0,NaN,1038.0,1029.0,1005.0,427.0,...,1132.0,1385.0,1333.0,1053.0,795.0,444.0,602.0,991.0,2350.0,789.0
5,La Rioja,986.0,340.0,814.0,927.0,1038.0,NaN,427.0,1063.0,659.0,...,330.0,600.0,533.0,283.0,435.0,640.0,834.0,311.0,2821.0,1311.0
6,Mendoza,985.0,466.0,1131.0,1269.0,1029.0,427.0,NaN,676.0,790.0,...,756.0,1023.0,957.0,152.0,235.0,775.0,586.0,713.0,2435.0,1019.0
7,Neuquén,989.0,907.0,1534.0,1690.0,1005.0,1063.0,676.0,NaN,1053.0,...,1370.0,1658.0,1591.0,824.0,643.0,1049.0,422.0,1286.0,1762.0,479.0
8,Paraná,375.0,348.0,500.0,656.0,427.0,659.0,790.0,1053.0,NaN,...,707.0,959.0,906.0,757.0,574.0,19.0,642.0,566.0,2635.0,1030.0
9,Posadas,834.0,919.0,291.0,263.0,857.0,1098.0,1384.0,1709.0,658.0,...,924.0,1007.0,992.0,1306.0,1200.0,664.0,1293.0,827.0,3207.0,1624.0


# exhaustivo

In [8]:
def generar_recorridos_posibles(cantidad_ciudades):
    '''
    componente para el método exhaustivo. Genera todos los recorridos posibles de longitud cantidad_ciudades
    siempre partiendo desde Bs. As. (para simplificar)
    '''
    aux = list(itertools.permutations([i for i in range(cantidad_ciudades)])) # generar permutaciones de las capitales
    
    recorridos_posibles = []
    for recorrido in aux:
        recorrido_aux = list(recorrido)   # itertools devuelve tuplas pero lo paso a listas
        recorrido_aux.append(recorrido[0]) # para que vuelva a la ciudad de partida
        recorridos_posibles.append(recorrido_aux)
    return recorridos_posibles

In [9]:
def calcular_distancia(tabla_distancias, recorrido):
    '''
    dado un recorrido [ciudad 1, ciudad 2,  .., ciudad n, ciudad 1], calcula la longitud del trayecto
    '''
    distancia_recorrida = 0
    for i in range(len(recorrido) - 1):
        distancia_actual = tabla_distancias[recorrido[i]][recorrido[i+1]]
        distancia_recorrida += distancia_actual
    return distancia_recorrida

In [10]:
def main_exhaustivo(tabla_distancias, cantidad_ciudades):

    distancia_minima = 9999999999
    t0 = time.perf_counter()
    recorridos_posibles = generar_recorridos_posibles(cantidad_ciudades)

    for recorrido in recorridos_posibles:
        distancia_recorrida = calcular_distancia(tabla_distancias, recorrido)
        if distancia_recorrida < distancia_minima:
            distancia_minima = distancia_recorrida
            recorrido_minimo = recorrido

    t1 = time.perf_counter()
    tiempo_ejecucion = t1 - t0

    return recorrido_minimo, distancia, tiempo_ejecucion

recorrido mínimo: [0, 1, 2, 3, 4, 0]
disntancia: 2501.0
